# Customer complaints analysis

In [6]:
# Importing python libraries to be used in the analysis process

import pandas as pd
import spacy
import gensim
import pyLDAvis
import numpy
import matplotlib
!python -m spacy download en_core_web_sm -qq
import emoji
import time

from gensim import corpora
from gensim.utils import simple_preprocess

import pandas as pd
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.decomposition import TruncatedSVD
# from sklearn.decomposition import LatentDirichletAllocation


[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


### Importing customer complaints csv file

In [7]:
customer_complaints = pd.read_csv("comcast_consumeraffairs_complaints.csv")

In [8]:
# Displaying the first five rows of the imported csv file

customer_complaints.head()

,author,posted_on,rating,text
0,"Alantae of Chesterfeild, MI","Nov. 22, 2016",1,I used to love Comcast. Until all these consta...
1,"Vera of Philadelphia, PA","Nov. 19, 2016",1,I'm so over Comcast! The worst internet provid...
2,"Sarah of Rancho Cordova, CA","Nov. 17, 2016",1,If I could give them a negative star or no sta...
3,"Dennis of Manchester, NH","Nov. 16, 2016",1,I've had the worst experiences so far since in...
4,"Ryan of Bellevue, WA","Nov. 14, 2016",1,Check your contract when you sign up for Comca...


In [9]:
# Filtering the complaints data to display only the elements that contain a complaint and removing the ones that are empty or null

customer_complaints = customer_complaints[pd.notnull(customer_complaints['text'])]


# Converting the 'text' column (containing text for customer complaints) to a list 

complaints_text_list = customer_complaints.text.values.tolist()


# Printing individual customer's complaints on a separate line 
iterator = 0
while len(complaints_text_list) > iterator:
    for individual_complaint in complaints_text_list:
    
    # Printing the complaint that previously contained an emoji, which emoji was removed as well as the complaint after the emoji was removed.
    
        if len(emoji.emoji_list(individual_complaint)) > 0:
            print(emoji.emoji_list(individual_complaint))
            print(emoji.replace_emoji(individual_complaint, replace=""))
            complaints_text_list[iterator] =  emoji.replace_emoji(individual_complaint, replace="")
            print(" ")
            print(" ")
            print("The list has been upadated.")
            print(" ")
            print(iterator, ": ", complaints_text_list[iterator])
            print(" ")
     
        iterator += 1
       

[{'match_start': 2022, 'match_end': 2023, 'emoji': '✌'}]
First off, I've been without internet for a week and on demand for a week as well. So I call the second day after nothing worked the first day to schedule a visit and the automated system kept saying that there was an outage in my area and it'll be resolved by 12:36. So I call later that afternoon and tell them and they try to send signals to the modem or whatever and it wouldn't work. So we scheduled a visit for that Friday between 10-12. Mind you that the first option they gave me was for the following freaking Monday and I said HECK NO that is way too long, I have a business to run.So the guy came on Friday at 1:30 NOT between 10-12 because I saw him pull up, he sat there for like 20 minutes and when I looked again he was freaking gone! So now I'm a little hot, so I call and ask what the heck is going on and why'd he just leave and the lady said, "Well there's another outage in your area and he wouldn't have been able to do an

### Preprocessing complaints text

In [36]:
# Loading nlp pipeline
nlp = spacy.load('en_core_web_sm')

# Defining a lemmatizer function

def lemmatizer(list):
    lemmatized_list = []
    iterator = 0
    
    while len(list) >= iterator:
        doc_w_lemma = [x.lemma_ for x in list]
        lemmatized_list.append(doc_w_lemma)       
        iterator += 1
    #print(lemmatized_list[0:11])
    return lemmatized_list

def text_preprocessor(list):
    processed_list_of_complaints = []
    index = 0
    while len(list) >= index:
        
        for x in list:
            x = x.lower()  # To convert all text to lowercase
            x = x.replace("!", " ").replace(".", " ").replace("?", " ") # To remove punctuation that is connected with words like "to?Do" etc.
            document = nlp(x) # Creating a document by running individual complaints through the nlp pipeline for tokenizing
            doc_no_punct = [token for token in document if token.is_punct == False ] # Creating a refined list of tokens from the 'document' tokens without any punctuation
            doc_no_stopwords = [token for token in doc_no_punct if token.is_stop == False]  # Further refining list of tokens from 'doc_no_punct' tokens without any stopwords
            lemmatized_doc = lemmatizer(doc_no_stopwords)
            processed_list_of_complaints.append(lemmatized_doc)
            # n_items = take(10, processed_dictionary_of_complaints.items())
            # print(n_items)
            
        if index < 10:
            print(index+1, " : ", processed_list_of_complaints[index])
        
        index += 1
        
    

list_tester = ["Terrible service!Will never go for comcast.", "Worth only $10?", "Going to throw out my subscriptions!", "Had the worst experience with comcast.", "Expensive and not good.", "Coverage is very low.", "frequently suffers from network disconnection.", "I have started looking for other service companies.", "Not recommended for anyone looking for a stable connection.", "Bought it last week, Not at all satisfied. ", "new to the service and already have complaints!", "Customer service is not at all cooperative.", "Has been weeks since last complaint was made still my complaint has not been addressed.", "Worst service not to be recommended to anyone.", "Money wasted for such an expensive service.", "Insufficient cabling", "Bad customer support", "No one picks up at call centre", "Looking for alternative better services", "Totally unsatisfied", "All claims made at the time of buying service not fulfilled", "Not good", "Will not recommend to anyone planning to work online", "was not that bad in the beginning, but now is the worst!"]
my_list = ['I went jogging', 'I tried', 'Throughly intestigate the case. It is sophisticated.']
start_time = time.time()
text_preprocessor(list_tester)
end_time = time.time()
time_diff = end_time - start_time
print("Time taken :  ", time_diff)

1  :  [['terrible', 'service', 'comcast'], ['terrible', 'service', 'comcast'], ['terrible', 'service', 'comcast'], ['terrible', 'service', 'comcast']]
2  :  [['worth', '$', '10'], ['worth', '$', '10'], ['worth', '$', '10'], ['worth', '$', '10']]
3  :  [['go', 'throw', 'subscription'], ['go', 'throw', 'subscription'], ['go', 'throw', 'subscription'], ['go', 'throw', 'subscription']]
4  :  [['bad', 'experience', 'comcast'], ['bad', 'experience', 'comcast'], ['bad', 'experience', 'comcast'], ['bad', 'experience', 'comcast']]
5  :  [['expensive', 'good'], ['expensive', 'good'], ['expensive', 'good']]
6  :  [['coverage', 'low'], ['coverage', 'low'], ['coverage', 'low']]
7  :  [['frequently', 'suffer', 'network', 'disconnection'], ['frequently', 'suffer', 'network', 'disconnection'], ['frequently', 'suffer', 'network', 'disconnection'], ['frequently', 'suffer', 'network', 'disconnection'], ['frequently', 'suffer', 'network', 'disconnection']]
8  :  [['start', 'look', 'service', 'company'], [

In [39]:

###########   -- Code that works --  ###########
# # # Setting the variable 'index' to 0 which will later on be incremented

index = 0

# Using the while loop to iterate through the complaints_text_list

while len(complaints_text_list) > index:

    for individual_complaint in complaints_text_list:
        
        individual_complaint = individual_complaint.lower()   # To convert all text to lowercase
       
        document = nlp(individual_complaint)   # Creating a document by running individual complaints through the nlp pipeline for tokenizing
         
        doc_no_punct = [token for token in document if not token.is_punct ]   # Creating a refined list of tokens from the 'document' tokens without any punctuation
        doc_no_stopwords = [token for token in doc_no_punct if token.is_stop == False]   # Further refining list of tokens from 'doc_no_punct' tokens without any stopwords
        #lemmatized_doc = [token.lemma_ for token in doc_no_stopwords]  # To lemmatize text like walking, resetting etc. but not working
        if index < 10:
            print("Tokens_of_complaint_",  index + 1 , ":    ", [token.text for token in doc_no_stopwords])   # Printing the tokens of each complaint as a list
            print(" ")
            print(" ")
        index += 1   # Incrementing index by 1

        
        
#########     --- code ends ---      #########


Tokens_of_complaint_ 1 :     ['love', 'comcast', 'constant', 'updates', 'internet', 'cable', 'crash', 'lot', 'night', 'day', 'channels', 'work', 'demand', 'play', 'wish', 'mins', 'ago', 'internet', 'crashed', '20', 'mins', 'reason', 'tired', 'thinking', 'switching', 'wow', 'xfinity']
 
 
Tokens_of_complaint_ 2 :     ['comcast', 'worst', 'internet', 'provider', 'taking', 'online', 'classes', 'multiple', 'times', 'late', 'assignments', 'power', 'interruptions', 'area', 'lead', 'poor', 'quality', 'internet', 'service', 'definitely', 'switching', 'verizon', 'pay', '$', '10', 'extra', 'dealing', 'w/', 'comcast', 'non', 'stopping', 'internet', 'problems']
 
 
Tokens_of_complaint_ 3 :     ['negative', 'star', 'stars', 'review', 'worked', 'industry', 'bad', 'customer', 'service', 'comcast', 'matter', 'money', 'afford', 'services', 'legitimate', 'ripoff', 'think', 'biggest', 'scam', 'mortgage', 'industry', 'major', 'meltdown', 'hope', 'comcast', 'exist', 'disregard', 'want', 'help', 'right', 't